# Install Packages

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
!pip3 install spacy
!python -m spacy download en_core_web_lg

In [3]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [6]:
import numpy as np
import pandas as pd
import io
import re
from pprint import pprint

# Retrieve and Convert File to Data Frame

In [8]:
directory = 'Desktop/SciFi_Texts/'

import pandas as pd
df = pd.read_csv('sci-fi_word_chunks_bag_of_words_output (1).csv')
df

,Unnamed: 0,Book + Chunk,Text
0,0,0 Chunk 0,1 And Ardrahan Ardrahan As As At By CHAPTER En...
1,1,0 Chunk 1,Afterwards And And And And And Anyway As Down ...
2,2,0 Chunk 2,And And Arriving As As But Come Curiously For ...
3,3,0 Chunk 3,2 A A All And And As Borderland Bps CHAPTER Cr...
4,4,0 Chunk 4,After An As Away Down For Further Gradually Ha...
...,...,...,...
8134,8134,0 Chunk 8134,Acentinol Agent Agent All And Below Below But ...
8135,8135,0 Chunk 8135,1 A Acentinol Agent All All And Archive Archiv...
8136,8136,0 Chunk 8136,A A ARE Acres Ahhh All And And Archive Archive...
8137,8137,0 Chunk 8137,A All All And And Anytime Archive Archive Arch...


In [9]:
#Drop any empty cells
df = df.dropna()
df

,Unnamed: 0,Book + Chunk,Text
0,0,0 Chunk 0,1 And Ardrahan Ardrahan As As At By CHAPTER En...
1,1,0 Chunk 1,Afterwards And And And And And Anyway As Down ...
2,2,0 Chunk 2,And And Arriving As As But Come Curiously For ...
3,3,0 Chunk 3,2 A A All And And As Borderland Bps CHAPTER Cr...
4,4,0 Chunk 4,After An As Away Down For Further Gradually Ha...
...,...,...,...
8134,8134,0 Chunk 8134,Acentinol Agent Agent All And Below Below But ...
8135,8135,0 Chunk 8135,1 A Acentinol Agent All All And Archive Archiv...
8136,8136,0 Chunk 8136,A A ARE Acres Ahhh All And And Archive Archive...
8137,8137,0 Chunk 8137,A All All And And Anytime Archive Archive Arch...


In [10]:
#Add values in Text column to new list (for further cleaning)
data = df.Text.values.tolist()

# Clean Texts

In [11]:
#Define list of stopwords
stop_words = stopwords.words('english')

# Add further stopwords by simply "appending" desired words to dictionary
stop_words.append('CHAPTER')

In [12]:
#Remove punctuation
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

<>:2: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \s
/var/folders/8k/6tq9yjnx511d5q4fkv00y_600000gn/T/ipykernel_9382/3478888187.py:2: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/8k/6tq9yjnx511d5q4fkv00y_600000gn/T/ipykernel_9382/3478888187.py:3: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [13]:
#Define function to perform simple preprocessing on text
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [14]:
#Run processing function on texts
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [15]:
#Define stopword removal
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]

#Define function to make bigrams
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#   return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Define function to lemmatize texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [16]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [18]:
#Run functions to remove stopwords, make bigrams, and lemmatize text
data_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_nostops)
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:4])

[['chapter', 'find', 'far', 'fuel', 'however', 'ireland_irish', 'irish', 'kraighten', 'kraighten', 'later', 'manuscript', 'possibly', 'right', 'tonnison', 'yet', 'abound', 'add_addresse', 'address', 'aid', 'alone', 'angler', 'appetite', 'around', 'arrive', 'ask', 'ask', 'assent', 'average_aware', 'away', 'away', 'awhile', 'back', 'bacon', 'bad', 'bare', 'base', 'begin', 'betray', 'blanket', 'bleak_blessing', 'bring', 'busy', 'called_calle', 'camp', 'car', 'cast', 'casually', 'chance_chance', 'clinging_close', 'close', 'colony', 'come', 'come', 'come', 'comer', 'come', 'comprehend', 'comrade', 'consult', 'contact', 'cottage', 'country', 'course', 'cover', 'crowd_crowd', 'curiously', 'cut', 'day', 'die', 'direction', 'discover', 'dismiss', 'doorway', 'driver_driver', 'driver', 'early', 'earth', 'elbow', 'elect', 'end', 'entirely', 'erect', 'escape', 'even', 'evening', 'ever', 'evident', 'exist', 'expression', 'eyed', 'face', 'faced_face', 'fact', 'fact', 'family', 'fall', 'fellow_fellow'

# Building Dictionary and Corpus

In [19]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Find Ideal # of Topics

In [20]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [21]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words, start=2, limit=40, step=6)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/megankane/opt/anaconda3/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/megankane/opt/anaconda3/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/gensim/__init__.py", line 11, in <module>
    from gensim import parsing, corpora, matutils, interfaces, models, similarities, utils  # noqa:F401
  File "/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/gensim/parsing/__init__.py", line 4, in <module>
    from .preprocessing import (  # noqa:F401
  File "/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/gensim/parsing/preprocessing.py", line 26, in <module>
    from gensim import utils
  File "/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/gensim/utils.py", line 36, in <module>
   

KeyboardInterrupt: 

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Create Topic Model with 20 Topics

In [22]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [23]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"faith_fame" + 0.000*"experienced_explore" + '
  '0.000*"increases_inevitable" + 0.000*"galactics_game" + 0.000*"flypaper" + '
  '0.000*"modernand" + 0.000*"climaxalien" + 0.000*"continuing_contrast" + '
  '0.000*"alarm_alcove" + 0.000*"envied_essential"'),
 (1,
  '0.015*"higgins_higgins" + 0.014*"station_station" + 0.010*"scrambled" + '
  '0.009*"plane_plane" + 0.006*"driver_driver" + 0.006*"mobile" + '
  '0.006*"paul_pauls" + 0.005*"gang" + 0.005*"jeep_jeep" + 0.004*"gut"'),
 (2,
  '0.005*"niche" + 0.004*"returned_revealed" + 0.002*"diabolical" + '
  '0.002*"genial" + 0.002*"beckoning" + 0.001*"speed_speeding" + '
  '0.001*"serenely" + 0.000*"slipping_slope" + 0.000*"kali_kali" + '
  '0.000*"dam_dam"'),
 (3,
  '0.000*"faith_fame" + 0.000*"experienced_explore" + '
  '0.000*"increases_inevitable" + 0.000*"galactics_game" + 0.000*"flypaper" + '
  '0.000*"modernand" + 0.000*"climaxalien" + 0.000*"continuing_contrast" + '
  '0.000*"alarm_alcove" + 0.000*"envied_essential"'),

In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.519557187409122

Coherence Score:  0.4020247749987556


In [25]:
vis = gensimvis.prepare(lda_model, corpus, id2word)

#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [26]:
pyLDAvis.display(vis)

TypeError: Object of type complex is not JSON serializable

# Compare: LDA Mallet Model

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
mallet_path = '/content/mallet-2.0.8/bin/mallet' 

In [ ]:
#Get ideal number of topics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#Get mallet path and run new topic model
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
#Create visualization
converted_ = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
vis_mallet = gensimvis.prepare(converted_, corpus, id2word)

In [ ]:
#Display visualization 
pyLDAvis.display(vis_mallet)

# Examine Topics Per Document

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()